In [33]:
!git clone https://github.com/eden-chung/RNEAPytorch.git

Cloning into 'RNEAPytorch'...
remote: Enumerating objects: 307, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 307 (delta 26), reused 46 (delta 15), pack-reused 236
Receiving objects: 100% (307/307), 743.70 KiB | 13.28 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [34]:
%cd RNEAPytorch

/content/RNEAPytorch/RNEAPytorch


In [35]:
from timeit import default_timer as timer
import torch
import numpy as np

In [36]:
from URDFParser import URDFParser
from URDFParser import Robot
from util import parseInputs, printUsage, validateRobot, initializeValues, printErr
from RBDReference import RBDReference
from GRiDCodeGenerator import GRiDCodeGenerator

In [37]:
# Store runtime results from 1000 trials, then take average for more accurate runtime
mxS_100iter = []
vxIv_100iter = []
fpass_100iter = []
bpass_100iter = []
RNEA_100iter = []

### Section 1: RNEA with PyTorch

In [38]:
parser = URDFParser()
robot = parser.parse('/content/RNEAPytorch/URDFParser/iiwa.urdf')

validateRobot(robot)

reference = RBDReference(robot)
#n is the number of joints
q, qd, u, n = initializeValues(robot, MATCH_CPP_RANDOM = True)

print("q", q)
print("qd", qd)
print("u", u)
print("n", n)

Link [base] does not have an origin. Assuming this is the fixed world base frame. Else there is an error with your URDF file.
Link [base] does not have inertial properties. Assuming this is the fixed world base frame. Else there is an error with your URDF file.
------------------------------------------
Assumed Input Joint Configuration Ordering
------------------------------------------
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
----------------------------
Total of n = 7 joints
----------------------------
q [-0.3369  1.2966 -0.6775 -1.4218 -0.7067 -0.135  -1.1495]
qd [ 0.433  -0.4216 -0.6454 -1.8605 -0.0131 -0.4583  0.7412]
u [ 0.7418  1.9284 -0.9039  0.0334  1.1799 -1.946   0.3287]
n 7


In [39]:
#initialize robot matrices & vectors
parent_id_arr = []
S_arr = []
Imat_arr = []
for ind in range(n):
  parent_id_arr.append(robot.get_parent_id(ind))
  S_arr.append(robot.get_S_by_id(ind).astype(np.float64))
  Imat_arr.append(robot.get_Imat_by_id(ind))

print("parent_id_arr", parent_id_arr)
parent_id_arr = np.array(parent_id_arr)
print("parent_id_arr shape:", parent_id_arr.shape)
S_arr = np.array(S_arr)
print("S_arr shape:", S_arr.shape)
print("S_arr", S_arr)
Imat_arr = np.array(Imat_arr)
print("Imat_arr shape:", Imat_arr.shape)
print("Imat_arr:", Imat_arr)

parent_id_arr [-1, 0, 1, 2, 3, 4, 5]
parent_id_arr shape: (7,)
S_arr shape: (7, 6)
S_arr [[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
Imat_arr shape: (7, 6, 6)
Imat_arr: [[[ 0.2091  0.      0.      0.     -0.6912 -0.1728]
  [ 0.      0.1989 -0.0003  0.6912  0.      0.    ]
  [ 0.     -0.0003  0.0227  0.1728  0.      0.    ]
  [ 0.      0.6912  0.1728  5.76    0.      0.    ]
  [-0.6912  0.      0.      0.      5.76    0.    ]
  [-0.1728  0.      0.      0.      0.      5.76  ]]

 [[ 0.0971 -0.     -0.      0.     -0.2667  0.3746]
  [-0.      0.0528 -0.      0.2667  0.     -0.0019]
  [-0.     -0.      0.0552 -0.3746  0.0019  0.    ]
  [ 0.      0.2667 -0.3746  6.35    0.      0.    ]
  [-0.2667  0.      0.0019  0.      6.35    0.    ]
  [ 0.3746 -0.0019  0.      0.      0.      6.35  ]]

 [[ 0.1496  0.      0.      0.     -0.455   0.105 ]
  [ 0.      0.1421  0.0003  0.455   0.      0. 

In [40]:
#write xmat functions to file
import array
import sys
import os
import inspect

for ind in range(n):
  with open(f'/content/xmat{ind}.py', 'w') as f:
      original_stdout = sys.stdout
      sys.stdout = f
      try:
          print("from numpy import array, sin, cos")
          print()
          content = robot.get_Xmat_Func_by_id(ind)
          source_code = inspect.getsource(content)
          print(source_code)
          # print(content)
      finally:
          sys.stdout = original_stdout
          f.close()

In [41]:
#store xmat functions into xmat array
py_file_location = "/content"
sys.path.append(os.path.abspath(py_file_location))
import xmat0, xmat1, xmat2, xmat3, xmat4, xmat5, xmat6

xmat_func_arr = []
xmat_func_arr.append(xmat0._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat1._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat2._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat3._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat4._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat5._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat6._lambdifygenerated(q[ind]))

xmat_func_arr = np.array(xmat_func_arr)
print("xmat_func_arr shape:", xmat_func_arr.shape)
print("xmat_func_arr", xmat_func_arr)

xmat_func_arr shape: (7, 6, 6)
xmat_func_arr [[[ 0.4089 -0.9126  0.      0.      0.      0.    ]
  [ 0.9126  0.4089  0.      0.      0.      0.    ]
  [ 0.      0.      1.      0.      0.      0.    ]
  [ 0.1437  0.0644  0.      0.4089 -0.9126  0.    ]
  [-0.0644  0.1437  0.      0.9126  0.4089  0.    ]
  [ 0.      0.      0.      0.      0.      1.    ]]

 [[-0.4089  0.     -0.9126  0.      0.      0.    ]
  [-0.9126  0.      0.4089  0.      0.      0.    ]
  [ 0.      1.      0.      0.      0.      0.    ]
  [ 0.     -0.0828  0.     -0.4089  0.     -0.9126]
  [ 0.     -0.1848  0.     -0.9126  0.      0.4089]
  [-0.2025  0.      0.      0.      1.      0.    ]]

 [[-0.4089  0.     -0.9126  0.      0.      0.    ]
  [-0.9126  0.      0.4089  0.      0.      0.    ]
  [ 0.      1.      0.      0.      0.      0.    ]
  [-0.1866  0.      0.0836 -0.4089  0.     -0.9126]
  [ 0.0836  0.      0.1866 -0.9126  0.      0.4089]
  [ 0.      0.      0.      0.      1.      0.    ]]

 [[ 0.4089  0

In [42]:
def cross_operator_batched(d_vec, d_output):

      d_output[0, 1, :] = -d_vec[2, :]
      d_output[0, 2, :] = d_vec[1, :]
      d_output[1, 0, :] = d_vec[2, :]
      d_output[1, 2, :] = -d_vec[0, :]
      d_output[2, 0, :] = -d_vec[1, :]
      d_output[2, 1, :] = d_vec[0, :]

      d_output[3, 1, :] = -d_vec[5, :]
      d_output[3, 2, :] = d_vec[4, :]
      d_output[3, 4, :] = -d_vec[2, :]
      d_output[3, 5, :] = d_vec[1, :]
      d_output[4, 0, :] = d_vec[5, :]
      d_output[4, 2, :] = -d_vec[3, :]
      d_output[4, 3, :] = d_vec[2, :]
      d_output[4, 5, :] = -d_vec[0, :]
      d_output[5, 0, :] = -d_vec[4, :]
      d_output[5, 1, :] = d_vec[3, :]
      d_output[5, 3, :] = -d_vec[1, :]
      d_output[5, 4, :] = d_vec[0, :]

In [43]:
#store xmat functions into xmat array
py_file_location = "/content"
sys.path.append(os.path.abspath(py_file_location))
import xmat0, xmat1, xmat2, xmat3, xmat4, xmat5, xmat6

xmat_func_arr = []
xmat_func_arr.append(xmat0._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat1._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat2._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat3._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat4._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat5._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat6._lambdifygenerated(q[ind]))

xmat_func_arr = np.array(xmat_func_arr)
print("xmat_func_arr shape:", xmat_func_arr.shape)
print("xmat_func_arr", xmat_func_arr)

xmat_func_arr shape: (7, 6, 6)
xmat_func_arr [[[ 0.4089 -0.9126  0.      0.      0.      0.    ]
  [ 0.9126  0.4089  0.      0.      0.      0.    ]
  [ 0.      0.      1.      0.      0.      0.    ]
  [ 0.1437  0.0644  0.      0.4089 -0.9126  0.    ]
  [-0.0644  0.1437  0.      0.9126  0.4089  0.    ]
  [ 0.      0.      0.      0.      0.      1.    ]]

 [[-0.4089  0.     -0.9126  0.      0.      0.    ]
  [-0.9126  0.      0.4089  0.      0.      0.    ]
  [ 0.      1.      0.      0.      0.      0.    ]
  [ 0.     -0.0828  0.     -0.4089  0.     -0.9126]
  [ 0.     -0.1848  0.     -0.9126  0.      0.4089]
  [-0.2025  0.      0.      0.      1.      0.    ]]

 [[-0.4089  0.     -0.9126  0.      0.      0.    ]
  [-0.9126  0.      0.4089  0.      0.      0.    ]
  [ 0.      1.      0.      0.      0.      0.    ]
  [-0.1866  0.      0.0836 -0.4089  0.     -0.9126]
  [ 0.0836  0.      0.1866 -0.9126  0.      0.4089]
  [ 0.      0.      0.      0.      1.      0.    ]]

 [[ 0.4089  0

In [44]:
# COMPARING to batched
batch_size = 100
h_vec_batched = np.ones((6,batch_size),  dtype=np.float64)
h_output_batched = np.zeros((6, 6, batch_size), dtype=np.float64)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
h_vec_batched = torch.as_tensor(h_vec_batched, device=device)
h_output_batched = torch.as_tensor(h_output_batched, device=device)

#on CPU
cross_operator_batched(h_vec_batched, h_output_batched) #warm-up once
print("cross operator output shape: ", h_output_batched.shape)
startnext = timer()
for i in range(100):
  cross_operator_batched(h_vec_batched, h_output_batched)
print("GPU Batched: " + str(timer() - startnext))

cross operator output shape:  torch.Size([6, 6, 100])
GPU Batched: 0.053458441999993056


In [45]:
def mxS_pytorch(S, vec, vec_output, mxS_output=None, alpha=None):
    # Input:
    # S, vec, vec_output, mxS_output, alpha are all torch tensor
    # returns the spatial cross product between vectors S and vec. vec=[v0, v1 ... vn] and S = [s0, s1, s2, s3, s4, s5]
    # derivative of spatial motion vector = v x m
    # print("vec shape: " + str(vec.shape))
    # print("vec_output shape: " + str(vec_output.shape))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    cross_operator_batched(vec, vec_output)

    mxS_output = alpha * torch.sum(vec_output * S, dim=1)

    return mxS_output.cpu().numpy()

In [46]:
# COMPARING to batched
batch_size = 100
# vec is a 6 by 1 matrix
h_vec_batched = np.ones((6,batch_size),  dtype=np.float64)

#S should be a 6 by 1 matrix
h_s_vec_batched = np.ones((6, 1, batch_size),  dtype=np.float64)

#vec output is a 6 by 6 matrix
h_output_batched = np.zeros((6, 6, batch_size), dtype=np.float64)

#mxS output is a 6 by 6 matrix
#TODO: mxS output should be (6, ) not (6, 6), need to change in CPU batched
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)

alpha = 0.1

# Convert input to torch tensor type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
h_vec_batched = torch.as_tensor(h_vec_batched, device=device)
h_s_vec_batched = torch.as_tensor(h_s_vec_batched, device=device)
h_output_batched = torch.as_tensor(h_output_batched, device=device)
h_mxS_output_batched = torch.as_tensor(h_mxS_output_batched, device=device)
alpha = torch.tensor(alpha, device=device)

startnext = timer()
mxS_pytorch(h_s_vec_batched, h_vec_batched, h_output_batched, h_mxS_output_batched, alpha) #warm-up once
totalTime = timer() - startnext
print("1 iteration: " + str(totalTime))

for i in range(1000):
  startnext = timer()
  mxS_pytorch(h_s_vec_batched, h_vec_batched, h_output_batched, h_mxS_output_batched, alpha)
  totalTime = timer() - startnext
  mxS_100iter.append(totalTime)
# print(mxS_pytorch(h_s_vec_batched, h_vec_batched, h_output_batched, h_mxS_output_batched, alpha).shape)

1 iteration: 0.0013574490000110018


In [47]:
def vxIv_pytorch(vec, Imat, res, batch_size):
    # Input:
    # vec, Imat, res, batch_size: should be of type torch tensors
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    temp = torch.sum(Imat * vec[:, None, :], dim=1)
    temp = temp.view(-1)

    vecXIvec = torch.zeros((6, batch_size), dtype=torch.float64, device=device)

    vecXIvec[0] = -vec[2]*temp[1]   +  vec[1]*temp[2] + -vec[2+3]*temp[1+3] +  vec[1+3]*temp[2+3]
    vecXIvec[1] =  vec[2]*temp[0]   + -vec[0]*temp[2] +  vec[2+3]*temp[0+3] + -vec[0+3]*temp[2+3]
    vecXIvec[2] = -vec[1]*temp[0]   +  vec[0]*temp[1] + -vec[1+3]*temp[0+3] +  vec[0+3]*temp[1+3]
    vecXIvec[3] = -vec[2]*temp[1+3] +  vec[1]*temp[2+3]
    vecXIvec[4] =  vec[2]*temp[0+3] + -vec[0]*temp[2+3]
    vecXIvec[5] = -vec[1]*temp[0+3] +  vec[0]*temp[1+3]
    res = vecXIvec

    return res.cpu().numpy()

In [48]:
# COMPARING to batched
batch_size = 100


h_vec_batched = np.ones((6, batch_size),  dtype=np.float64)
h_I_batched = np.ones((6, 6, batch_size),  dtype=np.float64)
h_output_batched = np.zeros((6, batch_size), dtype=np.float64)
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)

alpha = 0.1

# Convert input to torch tensor type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
h_vec_batched = torch.as_tensor(h_vec_batched, device=device)
h_I_batched= torch.as_tensor(h_I_batched, device=device)
h_output_batched = torch.as_tensor(h_output_batched, device=device)
h_mxS_output_batched = torch.as_tensor(h_mxS_output_batched, device=device)
alpha = torch.tensor(alpha, device=device)

#GPU
startnext = timer()
vxIv_pytorch(h_vec_batched, h_I_batched, h_output_batched, batch_size) #warm-up once
totalTime = timer() - startnext
print("1 iteration: " + str(totalTime))

# print("vxIV shape: ", h_output_batched.shape)
#testing in loop of 100
for i in range(1000):
  startnext = timer()
  vxIv_pytorch(h_vec_batched, h_I_batched, h_output_batched, batch_size)
  totalTime = timer() - startnext
  vxIv_100iter.append(totalTime)

1 iteration: 0.0008594369999741502


In [49]:
def rnea_fpass_pytorch(num_joints, parent_id_arr, xmat_func_arr, S_arr, Imat_arr, crOp_output, mxS_output, vxIv_output, batch_size, q, qd, qdd=None, GRAVITY=-9.81):
    """
    Forward Pass for RNEA algorithm. Computes the velocity and acceleration of each body in the tree necessary to produce a certain trajector

    OUTPUT:
    v : input qd is specifying value within configuration space with assumption of one degree of freedom.
    Output velocity is in general body coordinates and specifies motion in full 6 degrees of freedom
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    n = num_joints

    v = torch.zeros((6, n, batch_size), device=device)
    a = torch.zeros((6, n, batch_size), device=device)
    f = torch.zeros((6, n, batch_size), device=device)

    gravity_vec = torch.zeros((6, batch_size), device=device)
    gravity_vec[5, :] = -GRAVITY

    # parent_id_arr = torch.from_numpy(parent_id_arr).to(device)
    # xmat_func_arr = torch.from_numpy(xmat_func_arr).to(device)
    # S_arr = torch.from_numpy(S_arr).to(device)
    # Imat_arr = torch.from_numpy(Imat_arr).to(device)
    # crOp_output = torch.from_numpy(crOp_output).to(device)
    # mxS_output = torch.from_numpy(mxS_output).to(device)
    # vxIv_output = torch.from_numpy(vxIv_output).to(device)
    # q = torch.from_numpy(q).to(device)
    # qd = torch.from_numpy(qd).to(device)
    if qdd is not None:
        qdd = torch.from_numpy(qdd).to(device)

    for ind in range(n):
        parent_ind = parent_id_arr[ind]

        Xmat = xmat_func_arr[ind, :, :, :]
        S = S_arr[ind]

        if parent_ind == -1:
            a[:, ind, :] = torch.sum(Xmat * gravity_vec[:, None, :], dim=1)
        else:
            v[:, ind, :] = torch.sum(Xmat * v[:, parent_ind, :], dim=1)
            a[:, ind, :] = torch.sum(Xmat * a[:, parent_ind, :], dim=1)

        v[:,ind, :] = torch.add(v[:,ind, :], S*qd[ind])

        mxS_pytorch(S, v[:,ind, :], crOp_output, mxS_output, qd[ind])
        a[:, ind, :] = torch.add(a[:, ind, :], mxS_output)
        if qdd is not None:
            a[:,ind, :] = torch.add(a[:,ind, :], S*qdd[ind])

        Imat = Imat_arr[ind, :, :]

        temp = torch.sum(Imat*a[:, ind, :], dim=1)
        vxIv_pytorch(v[:,ind, :],Imat, vxIv_output, batch_size)
        f[:, ind, :] = torch.add(temp, vxIv_output)

        return (v,a,f)

In [50]:
batch_size = 10000


h_xmat_func_arr_batched = np.repeat(xmat_func_arr[:, :, :, np.newaxis], batch_size, axis=3)
h_S_arr_batched = np.repeat(S_arr[:, :, np.newaxis], batch_size, axis=2)

h_Imat_arr_batched = np.repeat(Imat_arr[:, :, :, np.newaxis], batch_size, axis=3)
h_q_batched = np.repeat(q[:, np.newaxis], batch_size, axis=1)
h_qd_batched = np.repeat(qd[:, np.newaxis], batch_size, axis=1)

h_crOp_output_batched = np.zeros((6, 6, batch_size), dtype=np.float64)
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)
h_vxIv_output_batched = np.zeros((6, batch_size), dtype=np.float64)

In [51]:
# Convert to tensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parent_id_arr = torch.from_numpy(parent_id_arr).to(device)
h_xmat_func_arr_batched = torch.from_numpy(h_xmat_func_arr_batched).to(device)
h_S_arr_batched = torch.from_numpy(h_S_arr_batched).to(device)
h_Imat_arr_batched = torch.from_numpy(h_Imat_arr_batched).to(device)
h_q_batched = torch.from_numpy(h_q_batched).to(device)
h_qd_batched = torch.from_numpy(h_qd_batched).to(device)
h_crOp_output_batched = torch.from_numpy(h_crOp_output_batched).to(device)
h_mxS_output_batched  = torch.from_numpy(h_mxS_output_batched ).to(device)
h_vxIv_output_batched = torch.from_numpy(h_vxIv_output_batched).to(device)

In [52]:
itr = 100

startnext = timer()
v,a,f = rnea_fpass_pytorch(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)
totalTime = timer() - startnext
print("1 iteration: " + str(totalTime))

for i in range(1000):
  startnext = timer()
  rnea_fpass_pytorch(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)
  totalTime = timer() - startnext
  fpass_100iter.append(totalTime)

1 iteration: 0.005052362000014909


In [53]:
def rnea_bpass_pytorch(S_arr, parent_id_arr, xmat_func_arr, q, qd, f, USE_VELOCITY_DAMPING = False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    n = len(q)
    c = torch.zeros((n, batch_size), device=device)

    for ind in range(n-1, -1, -1):
        S = S_arr[ind]
        c[ind, :] = torch.sum(S * f[:, ind, :], dim=0)

        parent_ind = parent_id_arr[ind]
        if parent_ind != -1:
            Xmat = xmat_func_arr[ind, :, :]
            temp = torch.sum(Xmat * f[:, ind, :], dim=1)

            # Ensure temp is correctly shaped for addition
            # temp = temp.transpose(0, 1)  # Swap the dimensions if necessary
            temp.unsqueeze(1)
            f[:, parent_ind, :] = torch.add(f[:, parent_ind, :], temp)

    return (c, f)

In [54]:
itr = 100
#for reference, batch_size is 10000 above
v,a,f = rnea_fpass_pytorch(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)

startnext = timer()
c, f = rnea_bpass_pytorch(h_S_arr_batched, parent_id_arr, h_xmat_func_arr_batched, h_q_batched, h_qd_batched, f, USE_VELOCITY_DAMPING = False)
totalTime = timer() - startnext
print("1 iteration: " + str(totalTime))

for i in range(1000):
  startnext = timer()
  rnea_bpass_pytorch(h_S_arr_batched, parent_id_arr, h_xmat_func_arr_batched, h_q_batched, h_qd_batched, f, USE_VELOCITY_DAMPING = False)
  totalTime = timer() - startnext
  bpass_100iter.append(totalTime)

1 iteration: 0.0022936770000114848


In [55]:
def rnea_pytorch(q, qd, qdd = None, GRAVITY = -9.81, USE_VELOCITY_DAMPING = False):

      # first do the forward pass
      v,a,f = rnea_fpass_pytorch(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)

      # then do the backward pass
      (c,f) = rnea_bpass_pytorch(h_S_arr_batched, parent_id_arr,h_xmat_func_arr_batched,
                               h_q_batched, h_qd_batched,
                               f, USE_VELOCITY_DAMPING = False)
      return (c,v,a,f)

In [56]:
for i in range(1000):
  startnext = timer()
  rnea_pytorch(q, qd, qdd = None, GRAVITY = -9.81, USE_VELOCITY_DAMPING = False)
  totalTime = timer() - startnext
  RNEA_100iter.append(totalTime)

In [57]:
mxS_100iter_avg = sum(mxS_100iter) / len(mxS_100iter)
vxIv_100iter_avg = sum(vxIv_100iter) / len(vxIv_100iter)
fpass_100iter_avg = sum(fpass_100iter) / len(fpass_100iter)
bpass_100iter_avg = sum(bpass_100iter) / len(bpass_100iter)
RNEA_100iter_avg = sum(RNEA_100iter) / len(RNEA_100iter)

In [58]:
mxS_100iter_avg

0.0005560175980001816

In [59]:
vxIv_100iter_avg

0.0006458105089998014

In [60]:
fpass_100iter_avg

0.002002759123999937

In [61]:
bpass_100iter_avg

0.0018331077699998274

In [62]:
RNEA_100iter_avg

0.00369316952400024